In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('stackexchange_812k_processed.csv')

In [3]:
data.columns

Index(['post_id', 'parent_id', 'comment_id', 'text', 'category',
       'text_processed', 'length', 'tokens'],
      dtype='object')

In [3]:
data['tokens']=data['tokens'].apply(lambda x: x.split())

In [4]:
data_train=data[data.category!='title']
data_train.shape

(704656, 8)

In [5]:
data_test=data[data.category=='title']
data_test.shape

(87354, 8)

In [7]:
data_test.head()

,post_id,parent_id,comment_id,text,category,text_processed,length,tokens
0,1,NaN,NaN,Eliciting priors from experts,title,eliciting priors from experts,4,"[eliciting, priors, from, experts]"
1,3,NaN,NaN,What are some valuable Statistical Analysis op...,title,what are some valuable statistical analysis op...,9,"[what, are, some, valuable, statistical, analy..."
2,4,NaN,NaN,Assessing the significance of differences in d...,title,assessing the significance of differences in d...,7,"[assessing, the, significance, of, differences..."
3,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title,the two cultures: statistics vs. machine learning,8,"[the, two, cultures:, statistics, vs., machine..."
4,7,NaN,NaN,Locating freely available data samples,title,locating freely available data samples,5,"[locating, freely, available, data, samples]"


In [6]:
from nltk.lm import MLE, Laplace
from nltk.lm import Vocabulary
from nltk.util import ngrams

ngrams_degree=3

In [37]:
train_data = [ngrams(t, n= ngrams_degree#, pad_right=True, 
                     #pad_left=True,left_pad_symbol="<s>", right_pad_symbol="</s>"
                    )
              for t in data_train.tokens.values]

In [10]:
list(train_data[0])

[('<s>', '<s>', 'how'),
 ('<s>', 'how', 'should'),
 ('how', 'should', 'i'),
 ('should', 'i', 'elicit'),
 ('i', 'elicit', 'prior'),
 ('elicit', 'prior', 'distributions'),
 ('prior', 'distributions', 'from'),
 ('distributions', 'from', 'experts'),
 ('from', 'experts', 'when'),
 ('experts', 'when', 'fitting'),
 ('when', 'fitting', 'a'),
 ('fitting', 'a', 'bayesian'),
 ('a', 'bayesian', 'model'),
 ('bayesian', 'model', '</s>'),
 ('model', '</s>', '</s>')]

In [49]:
training_data

<generator object padded_everygram_pipeline.<locals>.<genexpr> at 0x13fe99c50>

In [50]:
x=[i for i in training_data]

In [51]:
x

[]

In [25]:
train_data[0]

<generator object ngrams at 0x13f73cdd0>

In [38]:
test_data = [ngrams(t, n= ngrams_degree#, pad_right=True, 
                     #pad_left=True,left_pad_symbol="<s>", right_pad_symbol="</s>"
                   )
              for t in data_test.tokens.values]
#list(test_data[0])

In [39]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE,Laplace
from nltk.lm import Vocabulary


training_data, padded_vocab = padded_everygram_pipeline(ngrams_degree, train_data)
testing_data, _ = padded_everygram_pipeline(ngrams_degree, test_data)

In [11]:
training_data

<generator object padded_everygram_pipeline.<locals>.<genexpr> at 0x1ee5d77d0>

In [11]:
words = [word for sent in data_train.tokens.values for word in sent]
words.extend(["<s>", "</s>"])
vocab_reduced = Vocabulary(words, unk_cutoff = 20)
vocab=set(words)

In [17]:
'{}  {}'.format('<s>' in vocab_reduced, '<s>' in vocab)

'False  True'

In [12]:
# define the model
model = MLE(ngrams_degree)

In [17]:
model1 = MLE(ngrams_degree)
%time model1.fit(training_data, padded_vocab)

CPU times: user 32.8 s, sys: 292 ms, total: 33.1 s
Wall time: 33.5 s


In [40]:
model2 = MLE(ngrams_degree)
%time model2.fit(training_data, padded_vocab)

CPU times: user 1min 4s, sys: 1.57 s, total: 1min 6s
Wall time: 1min 6s


In [26]:
print(model1.generate(20, random_seed=7))

['</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']


In [13]:
%time model.fit(train_data, vocab)

CPU times: user 5min 21s, sys: 23.1 s, total: 5min 44s
Wall time: 5min 59s


In [41]:
print(model2.generate(20, random_seed=7))

['</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']


In [15]:
s = 0
for i, test in enumerate(testing_data):
    p = model.perplexity(test)
    s+=p
print(s)

inf


In [15]:
model.counts[['machine','learning']]['project']

10

In [23]:
model1.counts[['machine','learning']]['</s>']

0

In [47]:
model.generate(text_seed=['machine','learning'])

'process'

In [36]:
model1.generate(3, random_seed=7)

['</s>', '</s>', '</s>']

In [44]:
model2.generate(text_seed=['machine','learning'])

'</s>'

In [81]:
model2.perplexity([('machine','learning','model')])

inf

In [82]:
model2.counts[['machine','learning']]['model']

0

In [85]:
model2.score('with', ['machine','learning'])

0

In [163]:
'machine learning'.split()

['machine', 'learning']

In [106]:
padded_vocab['<s>']

1

In [114]:
train_data

<generator object padded_everygram_pipeline.<locals>.<genexpr> at 0x1fc172bd0>

In [119]:
Laplace(test_data, padded_vocab)

In [76]:
model2 = MLE(3)
model2.fit(training_data, padded_vocab)

In [75]:
model3 = Laplace(-1)
model3.fit(training_data, padded_vocab)

In [47]:
model3.perplexity([('machine','learning','model')])

534277.9999999999

In [71]:

s = 0
for i, test in enumerate(testing_data):
    p = model3.perplexity(test)
    s+=p
print(s)



KeyboardInterrupt: 

In [56]:
from sys import exc_info,stdout
import traceback
import cgitb
def longerror(func):
    '''
    错误调试长日志
    :param func:
    :return:
    '''
    def In(*vars):
        try :
            return func (*vars),
        except Exception as e :
            import cgitb
            cgitb.enable ( format = 'text' )
        return func (*vars),
    return In

def shorterror(func):
    '''
    错误调试短日志
    :param func:
    :return:
    '''
    def In(*vars):
        try :
            return func (*vars),
        except Exception as e :
            exc_type, exc_value, exc_traceback_obj = exc_info()
            traceback.print_exception(exc_type, exc_value, exc_traceback_obj, limit=2, file=stdout)
            print("exc_type: %s" % exc_type)
            print("exc_value: %s" % exc_value)
            print("exc_traceback_obj: %s" % exc_traceback_obj)
    return In

In [59]:
@shorterror
def dbugfun():
    s = 0
    for i, test in enumerate(test_data):
        p = model2.perplexity(test)
        s+=p
    
print(dbugfun())

NameError: name 'exc_info' is not defined

In [37]:
testing_data

<generator object padded_everygram_pipeline.<locals>.<genexpr> at 0x2c1299350>

In [237]:
import os
import requests
import io #codecs
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import Laplace, MLE
from nltk import word_tokenize, sent_tokenize 


textTest = "This is two banana. i like banana yes very much."
text = "This is an orange. This is a mango haha"

n = 2
# Tokenize the text.
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                for sent in sent_tokenize(text)]
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                for sent in sent_tokenize(textTest)]
test_data, _ = padded_everygram_pipeline(n, tokenized_text)

model = Laplace(-1) 
model.fit(train_data, padded_sents)

s = 0
for i, test in enumerate(test_data):
    p = model.perplexity(test)
    s += p

print ("Perplexity: {0}".format(s/(i+1)))

Perplexity: 11.830434183505876


[]

In [121]:
for test in train_data:
    print(model3.perplexity(test))

In [83]:
model2.perplexity('')

inf

In [82]:
#corpus_perplexity(df_test.text.values)
model2.perplexity(data_test.tokens.values)

TypeError: 'Laplace' object is not iterable

In [47]:
sentence = "the difference between the two approaches is discussed here"
print("[perplexity {:.2f}] {}".format(model2.perplexity(sentence), sentence))


[perplexity inf] the difference between the two approaches is discussed here


In [19]:
words[-3:]

['variable', '<s>', '</s>']

In [117]:
MLE_model()

In [ ]:
laplace()

In [245]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE,Laplace
from nltk.lm import Vocabulary
test_sentences = ['an ant']
test_tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in test_sentences]
test_data, _ = padded_everygram_pipeline(n, test_tokenized_text)
for test in test_data:
    print("per all", model.perplexity(test))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/bob/nltk_data'
    - '/Users/bob/opt/anaconda3/envs/nlpiaenv/nltk_data'
    - '/Users/bob/opt/anaconda3/envs/nlpiaenv/share/nltk_data'
    - '/Users/bob/opt/anaconda3/envs/nlpiaenv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [79]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Vocabulary
from nltk.tokenize import word_tokenize

train_sentences = ['an apple', 'an orange']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) for sent in train_sentences]

n = 2
train_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
words = [word for sent in tokenized_text for word in sent]
words.extend(["<s>", "</s>"])
padded_vocab = Vocabulary(words)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_sentences = ['an apple', 'an ant']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) for sent in test_sentences]

test_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
for i, test in enumerate(test_data):
      print("PP({0}):{1}".format(test_sentences[i], model.perplexity(test)))

MLE Estimates: [(('an', ('<s>',)), 1.0), (('apple', ('an',)), 0.5), (('</s>', ('apple',)), 1.0)]
MLE Estimates: [(('an', ('<s>',)), 1.0), (('ant', ('an',)), 0.0), (('</s>', ('ant',)), 0)]
PP(an apple):1.2599210498948732
PP(an ant):inf


In [77]:
test

<generator object bigrams at 0x12da49050>

In [76]:
tokenized_text

[['an', 'apple'], ['an', 'ant']]

In [ ]:
sentence = "the difference between the two approaches is discussed here"

In [75]:
words

['an', 'apple', 'an', 'orange', '<s>', '</s>']

In [68]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/bob/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [63]:
pip install nltk.tokenize.word_tokenize

ERROR: Could not find a version that satisfies the requirement nltk.tokenize.word_tokenize (from versions: none)
ERROR: No matching distribution found for nltk.tokenize.word_tokenize
Note: you may need to restart the kernel to use updated packages.


In [64]:
nltk.__version__

'3.4.5'